<a href="https://colab.research.google.com/github/edcalderin/BigDataEngineering_ICARO/blob/master/spark/practica_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalación

In [ ]:
%%bash
apt-get install openjdk-8-jdk-headless -qq > /dev/null
wget -q http://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
tar xf spark-2.4.7-bin-hadoop2.7.tgz
pip install -q findspark

In [ ]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-2.4.7-bin-hadoop2.7'
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Ejercicios

### FlatMap

In [ ]:
from pyspark.sql import Row

Ejercicio 1

In [ ]:
rdd = spark.sparkContext.parallelize([('hola'), ('curso'), ('datos')])
rdd.take(5)

['hola', 'curso', 'datos']

In [ ]:
rdd.map(lambda x: (x, len(x))).take(3)

[('hola', 4), ('curso', 5), ('datos', 5)]

In [ ]:
rdd.flatMap(lambda x: (x, len(x))).take(3)

['hola', 4, 'curso']

Ejercicio 2

In [ ]:
rdd = spark.sparkContext.parallelize([('hola'), ('curso'), ('datos')])
rdd.take(5)

['hola', 'curso', 'datos']

Ejercicio 3

In [ ]:
rdd = spark.sparkContext.parallelize([('hola mundo'), ('esto es el curso'), ('de data engineering')])
rdd.flatMap(lambda x:x.split(' ')).take(8)

['hola', 'mundo', 'esto', 'es', 'el', 'curso', 'de', 'data']

In [ ]:
rdd.map(lambda x: x.split(' ')).take(4)

[['hola', 'mundo'],
 ['esto', 'es', 'el', 'curso'],
 ['de', 'data', 'engineering']]

## Dataframe

In [ ]:
from pyspark.sql import Row
rdd = spark.sparkContext.parallelize([('Michael', None, 'M'), ('Andy', 30, 'F'), ('Justin', 19, 'M')])
row = Row('name', 'age', 'gender')
personaRdd = rdd.map(lambda x: row(x[0], x[1], x[2]))
personaRdd.take(3)

[Row(name='Michael', age=None, gender='M'),
 Row(name='Andy', age=30, gender='F'),
 Row(name='Justin', age=19, gender='M')]

In [ ]:
type(personaRdd)

pyspark.rdd.PipelinedRDD

In [ ]:
personasDF = personaRdd.toDF()
personasDF.show(5)

+-------+----+------+
|   name| age|gender|
+-------+----+------+
|Michael|null|     M|
|   Andy|  30|     F|
| Justin|  19|     M|
+-------+----+------+



Forzando el tipo de dato

In [ ]:
from pyspark.sql.types import StringType, IntegerType, StructField, StructType

scheme = StructType([
                     StructField('name', StringType(), False),
                     StructField('age', IntegerType()),
                     StructField('gender', StringType(), False)
            ])

In [ ]:
personasDF_2 = spark.createDataFrame(rdd, scheme)

In [ ]:
personasDF_2.show(3)

+-------+----+------+
|   name| age|gender|
+-------+----+------+
|Michael|null|     M|
|   Andy|  30|     F|
| Justin|  19|     M|
+-------+----+------+



Boston Dataset https://www.kaggle.com/puxama/bostoncsv

In [ ]:
boston_df = spark.read.csv('drive/MyDrive/Big_Data_Engineering/Boston.csv', header=True)

In [ ]:
boston_df.show(4, False)

+---+-------+---+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|_c0|crim   |zn |indus|chas|nox  |rm   |age |dis   |rad|tax|ptratio|black |lstat|medv|
+---+-------+---+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|1  |0.00632|18 |2.31 |0   |0.538|6.575|65.2|4.09  |1  |296|15.3   |396.9 |4.98 |24  |
|2  |0.02731|0  |7.07 |0   |0.469|6.421|78.9|4.9671|2  |242|17.8   |396.9 |9.14 |21.6|
|3  |0.02729|0  |7.07 |0   |0.469|7.185|61.1|4.9671|2  |242|17.8   |392.83|4.03 |34.7|
|4  |0.03237|0  |2.18 |0   |0.458|6.998|45.8|6.0622|3  |222|18.7   |394.63|2.94 |33.4|
+---+-------+---+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
only showing top 4 rows



In [ ]:
boston_df = boston_df.drop('_c0')

In [ ]:
boston_df.columns

['crim',
 'zn',
 'indus',
 'chas',
 'nox',
 'rm',
 'age',
 'dis',
 'rad',
 'tax',
 'ptratio',
 'black',
 'lstat',
 'medv']

In [ ]:
boston_df.printSchema()

root
 |-- crim: string (nullable = true)
 |-- zn: string (nullable = true)
 |-- indus: string (nullable = true)
 |-- chas: string (nullable = true)
 |-- nox: string (nullable = true)
 |-- rm: string (nullable = true)
 |-- age: string (nullable = true)
 |-- dis: string (nullable = true)
 |-- rad: string (nullable = true)
 |-- tax: string (nullable = true)
 |-- ptratio: string (nullable = true)
 |-- black: string (nullable = true)
 |-- lstat: string (nullable = true)
 |-- medv: string (nullable = true)



In [ ]:
print('Cantidad de filas', boston_df.count())
print('Cantidad de columnas', len(boston_df.columns))

Cantidad de filas 506
Cantidad de columnas 14


In [ ]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col
for column in boston_df.columns:
  boston_df = boston_df.withColumn(column,col(column).cast(DoubleType()))

In [ ]:
boston_df.select('crim', 'zn', 'age').where(col('age') > 50 ).show(5)

+-------+----+----+
|   crim|  zn| age|
+-------+----+----+
|0.00632|18.0|65.2|
|0.02731| 0.0|78.9|
|0.02729| 0.0|61.1|
|0.06905| 0.0|54.2|
|0.02985| 0.0|58.7|
+-------+----+----+
only showing top 5 rows



In [ ]:
boston_df.registerTempTable("boston_crimes")
spark.sql("SELECT crim, zn, age FROM boston_crimes WHERE age > 50 LIMIT 5").show()

+-------+----+----+
|   crim|  zn| age|
+-------+----+----+
|0.00632|18.0|65.2|
|0.02731| 0.0|78.9|
|0.02729| 0.0|61.1|
|0.06905| 0.0|54.2|
|0.02985| 0.0|58.7|
+-------+----+----+

